In [17]:
from scipy.signal import butter, lfilter
import fastdyn_fic_dmf as dmf
import numpy as np
import matplotlib.pyplot as plt
# Fetch default parameters
import tracemalloc
from scipy.io import loadmat
from scipy.stats import zscore, pearsonr
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
import os
import pickle as pkl
from joblib import Parallel, delayed
import mat73
def compute_fcd(data, wsize, overlap, isubdiag):
    T, N = data.shape
    win_start = np.arange(0, T - wsize - 1, wsize - overlap)
    nwins = len(win_start)
    fcd = np.zeros((len(isubdiag[0]), nwins))
    print(fcd.shape)
    print(data.shape)
    print((data[win_start[2]:win_start[2] + wsize + 1, :]).shape)
    for i in range(nwins):
        tmp = data[win_start[i]:win_start[i] + wsize + 1, :]
        cormat = np.corrcoef(tmp.T)
        fcd[:, i] = cormat[isubdiag[0],isubdiag[1]]
    return fcd

from scipy.signal import find_peaks
def get_peak_and_ms_rate(rates):
    """Get the value of the first peak from the autocorrealtion of the average firing rates"""
    signal = np.mean(rates, axis=0)
    signal -= np.mean(signal)

    # Calculate autocorrelation function
    autocorr = np.correlate(signal, signal, mode='full')

    # Normalize the autocorrelation function
    autocorr = autocorr / np.var(signal) / len(signal)
    autocorr = autocorr[len(signal)-1:]
    peaks, _ = find_peaks(autocorr, height=(0.2,0.8), prominence=0.2)
    if peaks.size==0:
        autocorr_value = 0
        time = 0
    else:
        autocorr_value = autocorr[peaks[0]]
        time = peaks[0]
    return autocorr_value,time

def get_peak_and_ms_bold(bold):
    """Get the value of the first peak from the autocorrealtion of the average  bold"""
    signal = np.mean(bold, axis=0)
    signal -= np.mean(signal)

    # Calculate autocorrelation function
    autocorr = np.correlate(signal, signal, mode='full')

    # Normalize the autocorrelation function
    autocorr = autocorr / np.var(signal) / len(signal)
    autocorr = autocorr[len(signal)-1:]
    peaks, _ = find_peaks(autocorr, height=(0.2,0.8), prominence=0.2)
    if peaks.size==0:
        autocorr_value = 0
        time = 0
    else:
        autocorr_value = autocorr[peaks[0]]
        time = peaks[0]
    return autocorr_value,time

In [18]:
data_struct_deep = mat73.loadmat('data/BOLD_timeseries_Deep.mat')['BOLD_timeseries_Deep']
data_struct_awake = mat73.loadmat('data/BOLD_timeseries_Awake.mat')['BOLD_timeseries_Awake']

In [19]:
NREGIONS = 200
data_deep = np.zeros((len(data_struct_deep), NREGIONS, data_struct_deep[0][0].shape[1]))
for i in range(len(data_struct_deep)):
    data_deep[i] = data_struct_deep[i][0][:NREGIONS, :]
# Z-score each region time series for each subject
data_zscored_deep = np.zeros(data_deep.shape)
for i in range(len(data_deep)):
    data_zscored_deep[i] = zscore(data_deep[i], axis=1)

In [20]:
data_awake = np.zeros((len(data_struct_awake), NREGIONS, data_struct_awake[0][0].shape[1]))
for i in range(len(data_struct_awake)):
    data_awake[i] = data_struct_awake[i][0][:NREGIONS, :]
# Z-score each region time series for each subject
data_zscored_awake = np.zeros(data_awake.shape)
for i in range(len(data_awake)):
    data_zscored_awake[i] = zscore(data_awake[i], axis=1)

In [24]:
# Compare the autocorrelation values between the two states
#autocorr_values_deep = np.zeros((len(data_zscored_deep), NREGIONS))
#autocorr_values_awake = np.zeros((len(data_zscored_awake), NREGIONS))
#times_deep = np.zeros((len(data_zscored_deep), NREGIONS))
#times_awake = np.zeros((len(data_zscored_awake), NREGIONS))

autocorr_values_deep = np.zeros((len(data_zscored_deep), ))
autocorr_values_awake = np.zeros((len(data_zscored_awake), ))
times_deep = np.zeros((len(data_zscored_deep), ))
times_awake = np.zeros((len(data_zscored_awake), ))

for i in range(len(data_zscored_deep)):
    #for j in range(NREGIONS):
    autocorr_values_deep[i],times_deep[i] = get_peak_and_ms_bold(data_zscored_deep[i,:,:])
for i in range(len(data_zscored_awake)):
    #for j in range(NREGIONS):
    autocorr_values_awake[i],times_awake[i] = get_peak_and_ms_bold(data_zscored_awake[i,:,:])

In [25]:
autocorr_values_awake

array([0.27825953, 0.25845929, 0.26384256, 0.24667986, 0.24521791,
       0.23626612, 0.22019747, 0.        , 0.21412076, 0.21933481,
       0.26632232, 0.21551434, 0.21135704, 0.22467154, 0.27665878,
       0.        ])

In [26]:
autocorr_values_deep

array([0.22560761, 0.        , 0.26207739, 0.25396531, 0.20332882,
       0.21517188, 0.        , 0.26432653, 0.21276896, 0.        ,
       0.        , 0.        , 0.        , 0.25440396, 0.22422646,
       0.        ])